In [1]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/regspace_analysis'

In [1]:
import numpy as np
import glob
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pyemma
import msmbuilder
from msmbuilder.msm import MarkovStateModel
from msmbuilder.lumping import PCCAPlus

import mdtraj as md

import glob
from msmbuilder.cluster import RegularSpatial

In [3]:
samples_200_p3_20_100per = glob.glob('../macro_sample/200_p3_20_100per/*.pdb')

In [4]:
samples_200_p3_50_100per = glob.glob('../macro_sample/200_p3_50_100per/*.pdb')

In [4]:
len(samples_200_p3_20_100per)

20

In [6]:
len(samples_200_p3_50_100per)

50

In [7]:
# samples_200_p3_20_100per

In [ ]:
samples_md = [md.load(x) for x in samples_200_p3_20_100per]

data = samples_md[0][0]

allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))
    
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)
    
regspace_objects_noNflank_allatoms_3 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_3.append(RegularSpatial(0.3, metric='rmsd'))
    regspace_objects_noNflank_allatoms_3[-1].fit(x)    

In [ ]:
total = 0
for x in regspace_objects_noNflank_allatoms_5:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

In [ ]:
total = 0
for x in regspace_objects_noNflank_allatoms_3:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

In [4]:
# save _5 centers to make .pse

In [ ]:
centers_5 = []

for x in regspace_objects_noNflank_allatoms_5:
    centers_5.append(x.cluster_center_indices_)
    
centers_5_ = []

for x in centers_5:
    centers_5_.append([])
    for y in x:
        centers_5_[-1].append(y[0])
        
centers_5_md = []

for i in range(len(centers_5_)):
    traj = samples_md[i][centers_5_[i][0]]
    if len(centers_5_[i]) > 1:
        for x in centers_5_[i][1:]:
            traj += samples_md[i][x]
    centers_5_md.append(traj)            

In [ ]:
for i in range(len(centers_5_md)):
    traj = centers_5_md[i]
    traj.save('../macro_sample/200_p3_20_100per/centers/%s.pdb' % i)  

In [4]:
from collections import Counter

glob_output = np.load('../glob_output.npy')
glob_output = [x.decode('UTF-8') for x in glob_output]

samples = np.load('../macro_sample/200_p3_20_100per/samples_200_p3_20_100per.npy')

run_results = []

for macrostate in samples:
    macrostate_results = []
    for frame in macrostate:
        x = glob_output[frame[0]]
        y = int(x.split('-')[0][9:])
        if x[:5] == '11707':
            if y <= 19:
                macrostate_results.append('1zkk')
            elif y in [20,21] or (y >= 42 and y <= 59):
                macrostate_results.append('4ij8')
            elif y in [22,25,26,27,28,31,32,33,34,35] or (y >= 60 and y <= 69):
                macrostate_results.append('Apo')
            elif y in [23,24,29,30,36,37,38,39,40,41] or y >= 70:
                macrostate_results.append('Inhibitor')
        elif x[:5] == '11709':
            if y in [0,2]:
                macrostate_results.append('1zkk_apo')
            elif y in [1,3]:
                macrostate_results.append('4ij8_apo')
    
    run_results.append(macrostate_results)
    
run_results_counters = []

for x in run_results:
    run_results_counters.append(Counter(x))    

In [8]:
run_results_counters

[Counter({'1zkk': 41, '4ij8': 29, 'Apo': 29, 'Inhibitor': 1}),
 Counter({'1zkk': 31, '4ij8': 34, 'Apo': 35}),
 Counter({'1zkk': 10,
          '1zkk_apo': 6,
          '4ij8': 20,
          '4ij8_apo': 39,
          'Apo': 7,
          'Inhibitor': 18}),
 Counter({'1zkk': 12,
          '1zkk_apo': 23,
          '4ij8': 4,
          '4ij8_apo': 29,
          'Apo': 15,
          'Inhibitor': 17}),
 Counter({'1zkk': 6,
          '1zkk_apo': 1,
          '4ij8': 12,
          '4ij8_apo': 19,
          'Apo': 1,
          'Inhibitor': 61}),
 Counter({'1zkk': 7, 'Apo': 18, 'Inhibitor': 75}),
 Counter({'1zkk': 24,
          '1zkk_apo': 26,
          '4ij8': 16,
          '4ij8_apo': 13,
          'Apo': 7,
          'Inhibitor': 14}),
 Counter({'1zkk': 53,
          '1zkk_apo': 9,
          '4ij8': 23,
          '4ij8_apo': 2,
          'Apo': 9,
          'Inhibitor': 4}),
 Counter({'1zkk': 6,
          '1zkk_apo': 25,
          '4ij8': 4,
          '4ij8_apo': 30,
          'Apo': 17,
     

In [5]:
regspace_5_cluster_center_run_of_origin = []

i = 0
for macrostate in regspace_objects_noNflank_allatoms_5:
    regspace_5_cluster_center_run_of_origin.append([])
    for center in macrostate.cluster_center_indices_:
        regspace_5_cluster_center_run_of_origin[-1].append(run_results[i][center[0]])
    i += 1    

In [6]:
regspace_5_cluster_center_run_of_origin

[['1zkk', 'Apo', '1zkk', '1zkk'],
 ['1zkk', 'Apo'],
 ['4ij8', '1zkk_apo', 'Apo', '1zkk', 'Inhibitor', '4ij8_apo'],
 ['1zkk_apo', '1zkk', '4ij8_apo', '1zkk_apo', '1zkk_apo'],
 ['1zkk', '1zkk', 'Inhibitor'],
 ['Apo',
  'Inhibitor',
  'Inhibitor',
  'Inhibitor',
  'Inhibitor',
  '1zkk',
  'Apo',
  'Inhibitor',
  '1zkk',
  'Inhibitor',
  'Inhibitor',
  'Inhibitor'],
 ['1zkk_apo',
  '4ij8',
  '1zkk',
  'Apo',
  '1zkk_apo',
  'Inhibitor',
  '4ij8',
  'Inhibitor',
  'Apo'],
 ['Apo', '1zkk'],
 ['Apo', '1zkk_apo', '1zkk', 'Inhibitor', '4ij8_apo', '1zkk'],
 ['4ij8', '1zkk_apo', '1zkk', 'Apo', '1zkk', 'Apo', '1zkk'],
 ['Apo', 'Inhibitor', '1zkk', 'Inhibitor', '1zkk_apo'],
 ['Apo', 'Apo', 'Apo', 'Inhibitor', 'Apo', '1zkk'],
 ['4ij8_apo', '4ij8_apo', '1zkk', 'Apo', 'Inhibitor', '1zkk_apo'],
 ['Apo',
  'Apo',
  'Inhibitor',
  '1zkk',
  '1zkk',
  'Inhibitor',
  '4ij8_apo',
  'Inhibitor',
  '4ij8',
  '1zkk_apo',
  '1zkk'],
 ['Inhibitor',
  '4ij8',
  'Inhibitor',
  'Apo',
  '1zkk',
  '4ij8_apo',
  '1zk

In [5]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))
    
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])
        
regspace_5_by_cluster_runs_of_origin = []

for i in range(len(regspace_frames)):
    clusters = regspace_frames[i]
    runs = run_results[i]
    
    regspace_5_by_cluster_runs_of_origin.append(dict())
    for j in range(len(clusters)):
        if clusters[j] not in regspace_5_by_cluster_runs_of_origin[-1]:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]] = [runs[j]]
        else:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]].append(runs[j])
            
regspace_5_by_cluster_runs_of_origin_counters = []

for macrostate in regspace_5_by_cluster_runs_of_origin:
    regspace_5_by_cluster_runs_of_origin_counters.append(dict())
    for cluster in macrostate:
        regspace_5_by_cluster_runs_of_origin_counters[-1][cluster] = Counter(macrostate[cluster])            

In [6]:
regspace_5_by_cluster_runs_of_origin_counters

[{0: Counter({'1zkk': 11, '4ij8': 24, 'Apo': 18, 'Inhibitor': 1}),
  1: Counter({'1zkk': 2, '4ij8': 2, 'Apo': 9}),
  2: Counter({'1zkk': 18}),
  3: Counter({'1zkk': 10, '4ij8': 3, 'Apo': 2})},
 {0: Counter({'1zkk': 31}), 1: Counter({'4ij8': 34, 'Apo': 35})},
 {0: Counter({'1zkk': 3, '4ij8': 18, '4ij8_apo': 7, 'Apo': 1, 'Inhibitor': 6}),
  1: Counter({'1zkk_apo': 5,
           '4ij8': 2,
           '4ij8_apo': 29,
           'Apo': 4,
           'Inhibitor': 9}),
  2: Counter({'Apo': 2}),
  3: Counter({'1zkk': 7}),
  4: Counter({'Inhibitor': 3}),
  5: Counter({'1zkk_apo': 1, '4ij8_apo': 3})},
 {0: Counter({'1zkk': 2,
           '1zkk_apo': 10,
           '4ij8_apo': 8,
           'Apo': 5,
           'Inhibitor': 5}),
  1: Counter({'1zkk': 9,
           '1zkk_apo': 6,
           '4ij8': 2,
           '4ij8_apo': 15,
           'Apo': 5,
           'Inhibitor': 9}),
  2: Counter({'1zkk_apo': 2, '4ij8': 1, '4ij8_apo': 3, 'Inhibitor': 2}),
  3: Counter({'1zkk': 1,
           '1zkk_apo': 1,

In [ ]:
# samples_200_p3_50_100per

In [11]:
samples_md = [md.load(x) for x in samples_200_p3_50_100per]

data = samples_md[0][0]

allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))
    
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)
    
regspace_objects_noNflank_allatoms_3 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_3.append(RegularSpatial(0.3, metric='rmsd'))
    regspace_objects_noNflank_allatoms_3[-1].fit(x)    

In [12]:
total = 0
for x in regspace_objects_noNflank_allatoms_5:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

4
4
4
2
2
2
3
5
4
4
1
3
2
4
3
4
3
4
5
1
1
2
2
1
2
10
4
3
4
5
4
7
6
4
4
2
3
3
4
4
4
4
5
9
9
4
4
5
6
16
Total: 205.000000


In [13]:
total = 0
for x in regspace_objects_noNflank_allatoms_3:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)

18
16
24
10
10
13
21
17
13
24
13
17
7
15
19
11
21
18
25
15
13
25
8
14
14
31
23
17
24
37
30
33
30
14
20
15
21
27
14
27
10
45
32
80
58
23
40
55
60
86
Total: 1253.000000


In [1]:
# now let's do unitime_2500

In [3]:
samples_unitime_2500_300_p1_20_100per = glob.glob('../macro_sample/unitime_2500_300_p1_20_100per/*.pdb')

In [4]:
samples_unitime_2500_300_p1_50_100per = glob.glob('../macro_sample/unitime_2500_300_p1_50_100per/*.pdb')

In [5]:
len(samples_unitime_2500_300_p1_20_100per)

20

In [6]:
len(samples_unitime_2500_300_p1_50_100per)

50

In [7]:
# samples_unitime_2500_300_p1_20_100per

In [ ]:
samples_md = [md.load(x) for x in samples_unitime_2500_300_p1_20_100per]

data = samples_md[0][0]

allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))
    
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)
    
regspace_objects_noNflank_allatoms_3 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_3.append(RegularSpatial(0.3, metric='rmsd'))
    regspace_objects_noNflank_allatoms_3[-1].fit(x)    

In [ ]:
total = 0
for x in regspace_objects_noNflank_allatoms_5:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

In [ ]:
total = 0
for x in regspace_objects_noNflank_allatoms_3:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

In [ ]:
centers_5 = []

for x in regspace_objects_noNflank_allatoms_5:
    centers_5.append(x.cluster_center_indices_)
    
centers_5_ = []

for x in centers_5:
    centers_5_.append([])
    for y in x:
        centers_5_[-1].append(y[0])
        
centers_5_md = []

for i in range(len(centers_5_)):
    traj = samples_md[i][centers_5_[i][0]]
    if len(centers_5_[i]) > 1:
        for x in centers_5_[i][1:]:
            traj += samples_md[i][x]
    centers_5_md.append(traj)            

In [10]:
for i in range(len(centers_5_md)):
    traj = centers_5_md[i]
    traj.save('../macro_sample/unitime_2500_300_p1_20_100per/centers/%s.pdb' % i)  

In [ ]:
from collections import Counter

glob_output = np.load('../glob_output.npy')
glob_output = [x.decode('UTF-8') for x in glob_output]

samples = np.load('../macro_sample/unitime_2500_300_p1_20_100per/samples_300_p1_20_100per.npy')

run_results = []

for macrostate in samples:
    macrostate_results = []
    for frame in macrostate:
        x = glob_output[frame[0]]
        y = int(x.split('-')[0][9:])
        if x[:5] == '11707':
            if y <= 19:
                macrostate_results.append('1zkk')
            elif y in [20,21] or (y >= 42 and y <= 59):
                macrostate_results.append('4ij8')
            elif y in [22,25,26,27,28,31,32,33,34,35] or (y >= 60 and y <= 69):
                macrostate_results.append('Apo')
            elif y in [23,24,29,30,36,37,38,39,40,41] or y >= 70:
                macrostate_results.append('Inhibitor')
        elif x[:5] == '11709':
            if y in [0,2]:
                macrostate_results.append('1zkk_apo')
            elif y in [1,3]:
                macrostate_results.append('4ij8_apo')
    
    run_results.append(macrostate_results)
    
run_results_counters = []

for x in run_results:
    run_results_counters.append(Counter(x))    

In [ ]:
run_results_counters

In [10]:
regspace_5_cluster_center_run_of_origin = []

i = 0
for macrostate in regspace_objects_noNflank_allatoms_5:
    regspace_5_cluster_center_run_of_origin.append([])
    for center in macrostate.cluster_center_indices_:
        regspace_5_cluster_center_run_of_origin[-1].append(run_results[i][center[0]])
    i += 1    

In [11]:
regspace_5_cluster_center_run_of_origin

[['Inhibitor'],
 ['Inhibitor', 'Inhibitor', 'Inhibitor'],
 ['Apo'],
 ['Inhibitor', '4ij8_apo'],
 ['1zkk_apo', 'Inhibitor'],
 ['4ij8', '4ij8'],
 ['1zkk_apo', 'Inhibitor', '1zkk_apo'],
 ['Inhibitor', '1zkk', 'Inhibitor', 'Apo', '1zkk_apo', '4ij8', 'Inhibitor'],
 ['1zkk', '1zkk_apo', '1zkk'],
 ['Inhibitor', '1zkk_apo', 'Inhibitor'],
 ['Inhibitor', 'Inhibitor', 'Apo', '1zkk'],
 ['Inhibitor', '1zkk', '4ij8_apo', '1zkk', 'Inhibitor', 'Inhibitor', '1zkk'],
 ['Inhibitor', 'Inhibitor', '1zkk'],
 ['4ij8_apo', 'Apo', '1zkk', '4ij8_apo', '4ij8', '1zkk_apo'],
 ['Inhibitor', '4ij8_apo', 'Apo', '1zkk_apo', 'Apo', 'Apo', 'Inhibitor'],
 ['1zkk',
  '1zkk',
  '4ij8_apo',
  '1zkk_apo',
  'Inhibitor',
  'Inhibitor',
  'Inhibitor',
  '4ij8_apo',
  '4ij8_apo'],
 ['1zkk_apo',
  'Inhibitor',
  'Apo',
  '4ij8_apo',
  '1zkk',
  '1zkk',
  'Inhibitor',
  '1zkk',
  'Apo',
  '1zkk',
  '1zkk_apo',
  '1zkk_apo'],
 ['1zkk_apo', 'Apo', 'Inhibitor', '1zkk'],
 ['4ij8',
  'Inhibitor',
  '1zkk',
  'Inhibitor',
  '4ij8',
  '

In [10]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))
    
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])
        
regspace_5_by_cluster_runs_of_origin = []

for i in range(len(regspace_frames)):
    clusters = regspace_frames[i]
    runs = run_results[i]
    
    regspace_5_by_cluster_runs_of_origin.append(dict())
    for j in range(len(clusters)):
        if clusters[j] not in regspace_5_by_cluster_runs_of_origin[-1]:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]] = [runs[j]]
        else:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]].append(runs[j])
            
regspace_5_by_cluster_runs_of_origin_counters = []

for macrostate in regspace_5_by_cluster_runs_of_origin:
    regspace_5_by_cluster_runs_of_origin_counters.append(dict())
    for cluster in macrostate:
        regspace_5_by_cluster_runs_of_origin_counters[-1][cluster] = Counter(macrostate[cluster])            

In [11]:
regspace_5_by_cluster_runs_of_origin_counters

[{0: Counter({'Inhibitor': 100})},
 {0: Counter({'1zkk': 1, 'Inhibitor': 78}),
  1: Counter({'Inhibitor': 19}),
  2: Counter({'Inhibitor': 2})},
 {0: Counter({'4ij8': 3, 'Apo': 97})},
 {0: Counter({'4ij8': 3, 'Inhibitor': 95}), 1: Counter({'4ij8_apo': 2})},
 {0: Counter({'1zkk_apo': 97, 'Inhibitor': 1}), 1: Counter({'Inhibitor': 2})},
 {0: Counter({'4ij8': 76}), 1: Counter({'4ij8': 24})},
 {0: Counter({'1zkk_apo': 54}),
  1: Counter({'1zkk_apo': 1, 'Inhibitor': 43}),
  2: Counter({'1zkk_apo': 2})},
 {0: Counter({'Apo': 7, 'Inhibitor': 22}),
  1: Counter({'1zkk': 13, 'Apo': 7, 'Inhibitor': 8}),
  2: Counter({'1zkk': 6, '4ij8': 1, 'Apo': 1, 'Inhibitor': 5}),
  3: Counter({'4ij8': 1, 'Apo': 8, 'Inhibitor': 2}),
  4: Counter({'1zkk_apo': 1}),
  5: Counter({'4ij8': 8, 'Inhibitor': 4}),
  6: Counter({'1zkk': 2, 'Apo': 3, 'Inhibitor': 1})},
 {0: Counter({'1zkk': 57, '4ij8': 2}),
  1: Counter({'1zkk': 4, '1zkk_apo': 29}),
  2: Counter({'1zkk': 8})},
 {0: Counter({'Inhibitor': 13}),
  1: Counte

In [11]:
# samples_unitime_2500_300_p1_50_100per

In [23]:
samples_md = [md.load(x) for x in samples_unitime_2500_300_p1_50_100per]

data = samples_md[0][0]

allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))
    
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)
    
regspace_objects_noNflank_allatoms_3 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_3.append(RegularSpatial(0.3, metric='rmsd'))
    regspace_objects_noNflank_allatoms_3[-1].fit(x)    

In [13]:
total = 0
for x in regspace_objects_noNflank_allatoms_5:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

1
1
1
2
1
1
2
1
3
2
1
5
2
1
1
2
2
2
1
1
1
1
3
14
5
2
4
2
1
1
3
2
1
5
5
2
8
3
1
4
4
4
7
7
9
10
9
11
19
6
Total: 187.000000


In [14]:
total = 0
for x in regspace_objects_noNflank_allatoms_3:
    print(x.n_clusters_)
    total += x.n_clusters_
print('Total: %f' % total)    

1
4
6
10
4
6
5
12
11
5
4
23
6
3
3
15
15
9
5
6
6
8
7
49
21
8
23
11
5
5
19
7
15
21
20
8
37
21
12
23
13
31
26
20
54
49
76
79
94
79
Total: 1000.000000


In [24]:
centers_5 = []

for x in regspace_objects_noNflank_allatoms_5:
    centers_5.append(x.cluster_center_indices_)
    
centers_5_ = []

for x in centers_5:
    centers_5_.append([])
    for y in x:
        centers_5_[-1].append(y[0])
        
centers_5_md = []

for i in range(len(centers_5_)):
    traj = samples_md[i][centers_5_[i][0]]
    if len(centers_5_[i]) > 1:
        for x in centers_5_[i][1:]:
            traj += samples_md[i][x]
    centers_5_md.append(traj)            

In [25]:
for i in range(len(centers_5_md)):
    traj = centers_5_md[i]
    traj.save('../macro_sample/unitime_2500_300_p1_50_100per/centers/%s.pdb' % i)  

In [26]:
from collections import Counter

glob_output = np.load('../glob_output.npy')
glob_output = [x.decode('UTF-8') for x in glob_output]

samples = np.load('../macro_sample/unitime_2500_300_p1_50_100per/samples_300_p1_50_100per.npy')

run_results = []

for macrostate in samples:
    macrostate_results = []
    for frame in macrostate:
        x = glob_output[frame[0]]
        y = int(x.split('-')[0][9:])
        if x[:5] == '11707':
            if y <= 19:
                macrostate_results.append('1zkk')
            elif y in [20,21] or (y >= 42 and y <= 59):
                macrostate_results.append('4ij8')
            elif y in [22,25,26,27,28,31,32,33,34,35] or (y >= 60 and y <= 69):
                macrostate_results.append('Apo')
            elif y in [23,24,29,30,36,37,38,39,40,41] or y >= 70:
                macrostate_results.append('Inhibitor')
        elif x[:5] == '11709':
            if y in [0,2]:
                macrostate_results.append('1zkk_apo')
            elif y in [1,3]:
                macrostate_results.append('4ij8_apo')
    
    run_results.append(macrostate_results)
    
run_results_counters = []

for x in run_results:
    run_results_counters.append(Counter(x))    

In [27]:
run_results_counters

[Counter({'4ij8_apo': 5, 'Apo': 95}),
 Counter({'Inhibitor': 100}),
 Counter({'1zkk': 100}),
 Counter({'4ij8': 98, 'Inhibitor': 2}),
 Counter({'4ij8': 94, '4ij8_apo': 6}),
 Counter({'1zkk': 97, 'Apo': 3}),
 Counter({'1zkk_apo': 1, '4ij8_apo': 95, 'Inhibitor': 4}),
 Counter({'Inhibitor': 100}),
 Counter({'1zkk_apo': 96, '4ij8_apo': 3, 'Apo': 1}),
 Counter({'Inhibitor': 100}),
 Counter({'4ij8': 1, 'Apo': 99}),
 Counter({'1zkk': 14, '1zkk_apo': 32, '4ij8': 22, 'Inhibitor': 32}),
 Counter({'1zkk': 1, 'Apo': 5, 'Inhibitor': 94}),
 Counter({'Inhibitor': 100}),
 Counter({'4ij8': 3, 'Inhibitor': 97}),
 Counter({'1zkk': 4,
          '1zkk_apo': 2,
          '4ij8': 56,
          '4ij8_apo': 33,
          'Apo': 3,
          'Inhibitor': 2}),
 Counter({'1zkk': 12, 'Apo': 87, 'Inhibitor': 1}),
 Counter({'1zkk_apo': 98, 'Inhibitor': 2}),
 Counter({'4ij8': 99, 'Apo': 1}),
 Counter({'Inhibitor': 100}),
 Counter({'1zkk': 37, 'Apo': 62, 'Inhibitor': 1}),
 Counter({'1zkk': 9, 'Apo': 91}),
 Counter({'4i

In [1]:
# ok back here to correct Shi analysis heatmaps for 1000 and 2500

In [3]:
samples_200_p3_20_100per = glob.glob('../macro_sample/200_p3_20_100per/*.pdb')

In [4]:
samples_md = [md.load(x) for x in samples_200_p3_20_100per]

In [5]:
data = samples_md[0][0]

In [6]:
allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))

In [7]:
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)

In [8]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))

In [9]:
from collections import Counter

In [10]:
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])

In [11]:
len(regspace_frames)

20

In [12]:
regspace_counters = []

for x in regspace_frames:
    regspace_counters.append(Counter(x))

In [13]:
msm_populations = np.array([ 0.00937809,  0.00511237,  0.00532692,  0.03194538,  0.01459391,
        0.01911732,  0.0277873 ,  0.00540315,  0.05325258,  0.01245424,
        0.02232931,  0.01905987,  0.01439557,  0.17017188,  0.08234085,
        0.04125468,  0.07237103,  0.21521267,  0.0586633 ,  0.11982958])

In [14]:
shi_data = np.array([
[[],['2-1, 2-3, 2-5, 5-5, 13-5'],['5-12, 9-7'],['5-4'],[],['5-7, 6-9'],['6-1, 6-6'],['5-8, 5-10, 6-8, 13-7, 14-5']],
[[],['2-6, 3-3, 3-5, 8-5, 11-1, 11-3, 11-5'],['2-2, 2-4, 5-1, 5-2, 8-2, 8-4, 9-2, 9-3, 10-2'],['3-1, 10-3, 13-8, 14-7'],['9-5'],['5-11, 6-3, 14-2'],['6-2'],['6-7, 14-8']],
[[],['0-2, 3-4, 5-9, 8-1, 8-3, 10-4, 11-4, 13-1, 13-2, 13-10, 16-4'],['3-2, 5-6, 10-1, 16-6'],['0-1, 0-4, 13-9'],[],['0-3'],[],[]],
[['5-3'],['8-6, 11-6, 13-11, 16-9'],['1-2, 9-4, 15-7'],['13-3, 13-4, 18-5'],[],['1-1, 6-4'],['14-1, 14-10'],['6-5, 14-4']],
[[],['4-2, 4-3, 7-2, 9-1, 9-6, 10-5, 13-6, 14-9, 16-10, 18-3, 19-3'],['4-1, 16-1, 16-7, 18-1, 19-1, 19-10'],['11-2, 12-1, 12-2, 12-3, 12-4, 12-6, 14-3, 15-2, 15-6, 16-2, 16-5, 18-4, 18-6, 19-7, 19-9'],[],['7-1, 15-9, 15-10, 15-11, 17-5, 18-7, 19-6'],['17-1'],['14-6, 15-5, 17-2']],
[[],['12-5, 15-4, 15-8, 16-8, 18-2, 19-2, 19-4, 19-8, 19-11'],[],['15-1, 15-3, 16-3, 17-4'],[],[],[],['17-3, 19-5']],
[[],[],[],[],[],[],[],[]],
])         

In [15]:
shi_data2 = np.empty((7,8)).tolist()

for i in range(7):
    for j in range(8):
        if shi_data[i,j] == []:
            shi_data2[i][j] = []
        else:    
            shi_data2[i][j] = shi_data[i,j][0].split(', ')
            
shi_data2 = np.asarray(shi_data2)            

In [16]:
shi_data = shi_data2

In [17]:
shi_data_corrected = np.zeros((7,8))

In [18]:
for i in range(7):
    for j in range(8):
        data = shi_data[i,j]
        data_split = [x.split('-') for x in data]
        result = 0
        for y in data_split:
            result += msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values())))
        shi_data_corrected[i,j] = result

In [19]:
np.sum(shi_data_corrected)

1.0

In [ ]:
# hmmm double check this - pretty low

In [ ]:
# double checking

In [84]:
n = 0

for x in shi_data:
    for y in x:
        n += len(y)
        
n        

138

In [86]:
regspace_objects_noNflank_allatoms_5[0]

RegularSpatial(d_min=0.5, metric='rmsd')

In [88]:
regspace_objects_noNflank_allatoms_5[0].n_clusters_

4

In [89]:
n_clusters = 0

for x in regspace_objects_noNflank_allatoms_5:
    n_clusters += x.n_clusters_
    
n_clusters    

138

In [20]:
shi_data

array([[[], ['2-1', '2-3', '2-5', '5-5', '13-5'], ['5-12', '9-7'], ['5-4'],
        [], ['5-7', '6-9'], ['6-1', '6-6'],
        ['5-8', '5-10', '6-8', '13-7', '14-5']],
       [[], ['2-6', '3-3', '3-5', '8-5', '11-1', '11-3', '11-5'],
        ['2-2', '2-4', '5-1', '5-2', '8-2', '8-4', '9-2', '9-3', '10-2'],
        ['3-1', '10-3', '13-8', '14-7'], ['9-5'], ['5-11', '6-3', '14-2'],
        ['6-2'], ['6-7', '14-8']],
       [[],
        ['0-2', '3-4', '5-9', '8-1', '8-3', '10-4', '11-4', '13-1', '13-2', '13-10', '16-4'],
        ['3-2', '5-6', '10-1', '16-6'], ['0-1', '0-4', '13-9'], [],
        ['0-3'], [], []],
       [['5-3'], ['8-6', '11-6', '13-11', '16-9'], ['1-2', '9-4', '15-7'],
        ['13-3', '13-4', '18-5'], [], ['1-1', '6-4'], ['14-1', '14-10'],
        ['6-5', '14-4']],
       [[],
        ['4-2', '4-3', '7-2', '9-1', '9-6', '10-5', '13-6', '14-9', '16-10', '18-3', '19-3'],
        ['4-1', '16-1', '16-7', '18-1', '19-1', '19-10'],
        ['11-2', '12-1', '12-2', '12-3', '1

In [97]:
# fixed! going back to dih to review numbers there - could have also made a mistake

In [98]:
plt.figure()
plt.imshow(shi_data_corrected)
plt.colorbar()
plt.savefig('shi_data_corrected_1000_20.png', dpi=300)
plt.close()

In [187]:
plt.figure()
plt.imshow(shi_data_corrected, cmap='Blues')
plt.colorbar()
plt.savefig('shi_data_corrected_1000_20_blue.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [188]:
plt.figure()
plt.imshow(shi_data_corrected, cmap='Reds')
plt.colorbar()
plt.savefig('shi_data_corrected_1000_20_red.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [21]:
plt.figure()
plt.imshow(np.log10(shi_data_corrected), cmap='Reds')
plt.colorbar()
plt.savefig('shi_data_corrected_1000_20_red_log10.png', dpi=300)
plt.close()

In [22]:
plt.figure()
plt.imshow(np.log10(shi_data_corrected), cmap='Blues')
plt.colorbar()
plt.savefig('shi_data_corrected_1000_20_blue_log10.png', dpi=300)
plt.close()

In [99]:
shi_data_corrected

array([[ 0.        ,  0.01461632,  0.00208547,  0.00152939,  0.        ,
         0.00104257,  0.00472384,  0.00656187],
       [ 0.        ,  0.01776357,  0.05692239,  0.02961796,  0.00074725,
         0.00321714,  0.0102813 ,  0.0019349 ],
       [ 0.        ,  0.10259267,  0.02679458,  0.01157604,  0.        ,
         0.00168806,  0.        ,  0.        ],
       [ 0.00516168,  0.00699405,  0.0078009 ,  0.05404291,  0.        ,
         0.01158826,  0.04034702,  0.02992058],
       [ 0.        ,  0.02689855,  0.13989006,  0.08992482,  0.        ,
         0.02739951,  0.1205191 ,  0.0175368 ],
       [ 0.        ,  0.05116983,  0.        ,  0.01350067,  0.        ,
         0.        ,  0.        ,  0.06360997],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]])

In [23]:
# make a pie chart

In [24]:
shi_data_flatten = shi_data.flatten()

regspace_counters_struct_macro = []

for x in range(len(regspace_counters)):
    new_dict = dict()
    macrostate = regspace_counters[x]
    for y in macrostate.keys():
        for i in range(len(shi_data_flatten)):
            data = shi_data_flatten[i]
            if '%s-%s' % (x,y+1) in data:
                new_key = i
                break
        if not new_key in new_dict:        
            new_dict[new_key] = macrostate[y]
        else:
            new_dict[new_key] += macrostate[y]
    regspace_counters_struct_macro.append(new_dict)    
    
regspace_counters_struct_macro = []

for x in range(len(regspace_counters)):
    new_dict = dict()
    macrostate = regspace_counters[x]
    for y in macrostate.keys():
        for i in range(len(shi_data_flatten)):
            data = shi_data_flatten[i]
            if '%s-%s' % (x,y+1) in data:
                new_key = i
                break
        if not new_key in new_dict:        
            new_dict[new_key] = macrostate[y]
        else:
            new_dict[new_key] += macrostate[y]
    regspace_counters_struct_macro.append(new_dict)        

In [25]:
regspace_counters_struct_macro

[{17: 13, 19: 69, 21: 18},
 {26: 69, 29: 31},
 {1: 40, 9: 4, 10: 56},
 {9: 17, 11: 30, 17: 7, 18: 46},
 {33: 14, 34: 86},
 {1: 3, 2: 7, 3: 8, 5: 4, 7: 5, 10: 31, 13: 1, 17: 5, 18: 9, 24: 27},
 {5: 1, 6: 17, 7: 2, 13: 2, 14: 37, 15: 4, 29: 36, 31: 1},
 {33: 40, 37: 60},
 {9: 2, 10: 79, 17: 14, 25: 5},
 {2: 6, 10: 28, 12: 6, 26: 31, 33: 29},
 {10: 11, 11: 29, 17: 7, 18: 40, 33: 13},
 {9: 58, 17: 10, 25: 10, 35: 22},
 {35: 99, 41: 1},
 {1: 7, 7: 2, 11: 7, 17: 50, 19: 3, 25: 1, 27: 29, 33: 1},
 {7: 2, 11: 2, 13: 3, 15: 1, 30: 49, 31: 36, 33: 1, 35: 4, 39: 2},
 {26: 1, 35: 44, 37: 9, 39: 2, 41: 20, 43: 24},
 {17: 3, 18: 2, 25: 1, 33: 4, 34: 60, 35: 24, 41: 4, 43: 2},
 {37: 7, 38: 56, 39: 7, 43: 1, 47: 29},
 {27: 8, 33: 2, 34: 45, 35: 23, 37: 1, 41: 21},
 {33: 8, 34: 48, 35: 16, 37: 4, 41: 23, 47: 1}]

In [26]:
# so we're doing another change - let's have a palette of 56 colors so every structural macrostate has its own constant
# color also between all models so we can do a direct comparison

In [27]:
colors = ["#aa5f5d",
"#4ecb5f",
"#c45fe3",
"#87bc2d",
"#8b39b6",
"#379f3b",
"#9d5ee0",
"#b9bd40",
"#4c69db",
"#d0b231",
"#8163d6",
"#86be59",
"#ac36a2",
"#678f24",
"#c35ec4",
"#58c086",
"#d13f94",
"#448136",
"#ed77d7",
"#958f24",
"#c07de7",
"#e4a231",
"#704ca8",
"#e6832c",
"#4789dc",
"#e1532f",
"#48b1da",
"#d93640",
"#52bfb0",
"#d3335a",
"#37835d",
"#e3427e",
"#a1bb74",
"#b4509d",
"#b9a753",
"#8f83de",
"#c38734",
"#5666a4",
"#d56c35",
"#96a0e0",
"#a64219",
"#c88acd",
"#5a6322",
"#895191",
"#838948",
"#ac5487",
"#88671f",
"#e488b1",
"#d8af70",
"#8e415d",
"#eb9a6f",
"#be506e",
"#8e552f",
"#e88280",
"#b9744a",
"#ba4846"]

In [31]:
dictionary_of_colors = dict()

for i in range(56):
    dictionary_of_colors[i] = colors[i]

colors2 = []

i = 0
for x in regspace_counters_struct_macro:
    colors2.append([])
    for key in x:
        color_ = dictionary_of_colors[key]
        colors2[-1].append(color_)    

In [35]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("piecharts/model4_piecharts_colorperstrmacro.pdf")

for i in range(len(regspace_counters_struct_macro)):
    x = regspace_counters_struct_macro[i]
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%', colors=colors2[i])
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [26]:
# runs of origin

In [27]:
from collections import Counter

glob_output = np.load('../glob_output.npy')
glob_output = [x.decode('UTF-8') for x in glob_output]

samples = np.load('../macro_sample/200_p3_20_100per/samples_200_p3_20_100per.npy')

run_results = []

for macrostate in samples:
    macrostate_results = []
    for frame in macrostate:
        x = glob_output[frame[0]]
        y = int(x.split('-')[0][9:])
        if x[:5] == '11707':
            if y <= 19:
                macrostate_results.append('1zkk')
            elif y in [20,21] or (y >= 42 and y <= 59):
                macrostate_results.append('4ij8')
            elif y in [22,25,26,27,28,31,32,33,34,35] or (y >= 60 and y <= 69):
                macrostate_results.append('Apo')
            elif y in [23,24,29,30,36,37,38,39,40,41] or y >= 70:
                macrostate_results.append('Inhibitor')
        elif x[:5] == '11709':
            if y in [0,2]:
                macrostate_results.append('1zkk_apo')
            elif y in [1,3]:
                macrostate_results.append('4ij8_apo')
    
    run_results.append(macrostate_results)
    
run_results_counters = []

for x in run_results:
    run_results_counters.append(Counter(x))    

In [28]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))
    
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])
        
regspace_5_by_cluster_runs_of_origin = []

for i in range(len(regspace_frames)):
    clusters = regspace_frames[i]
    runs = run_results[i]
    
    regspace_5_by_cluster_runs_of_origin.append(dict())
    for j in range(len(clusters)):
        if clusters[j] not in regspace_5_by_cluster_runs_of_origin[-1]:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]] = [runs[j]]
        else:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]].append(runs[j])
            
regspace_5_by_cluster_runs_of_origin_counters = []

for macrostate in regspace_5_by_cluster_runs_of_origin:
    regspace_5_by_cluster_runs_of_origin_counters.append(dict())
    for cluster in macrostate:
        regspace_5_by_cluster_runs_of_origin_counters[-1][cluster] = Counter(macrostate[cluster])            

In [29]:
shi_data_runs_origin = [
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
]

for i in range(7):
    for j in range(8):
        data = shi_data[i,j]
        data_split = [x.split('-') for x in data]
        output_dict = shi_data_runs_origin[i][j]
        for y in data_split:
            weight = msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values())))
            weight = weight / np.sum([msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values()))) for y in data_split])
            counter = regspace_5_by_cluster_runs_of_origin_counters[int(y[0])][int(y[1])-1]
            temp_dict = dict()
            for run_of_origin in counter:
                temp_dict[run_of_origin] = counter[run_of_origin] / np.sum([counter[x] for x in counter])
            
            for x in temp_dict:
                temp_dict[x] = temp_dict[x] * weight
                if x not in output_dict:
                    output_dict[x] = temp_dict[x]
                else:
                    output_dict[x] += temp_dict[x]

In [30]:
shi_data_runs_origin

[[{},
  {'1zkk': 0.24378539844696334,
   '4ij8': 0.18202697707915411,
   '4ij8_apo': 0.025511511817558005,
   'Apo': 0.12735945175581553,
   'Inhibitor': 0.42131666090050912},
  {'1zkk': 0.35831517432931559, 'Inhibitor': 0.64168482567068441},
  {'Apo': 0.125, 'Inhibitor': 0.875},
  {},
  {'Apo': 1.0},
  {'1zkk': 0.17647058823529413,
   '1zkk_apo': 0.41176470588235298,
   '4ij8': 0.17647058823529413,
   '4ij8_apo': 0.058823529411764705,
   'Inhibitor': 0.17647058823529413},
  {'1zkk': 0.38481844480044741,
   '1zkk_apo': 0.12548388289393142,
   '4ij8_apo': 0.25933456190651599,
   'Inhibitor': 0.23036311039910504}],
 [{},
  {'1zkk': 0.16094630112478076,
   '1zkk_apo': 0.14087923419167214,
   '4ij8': 0.046697055382050615,
   '4ij8_apo': 0.11090951121778035,
   'Apo': 0.30073528135192834,
   'Inhibitor': 0.23983261673178752},
  {'1zkk': 0.080716412034104568,
   '1zkk_apo': 0.21207997367798689,
   '4ij8': 0.048044557029777349,
   '4ij8_apo': 0.25777660591326607,
   'Apo': 0.17520233807313576

In [45]:
# now 2500

In [3]:
samples_unitime_2500_300_p1_20_100per = glob.glob('../macro_sample/unitime_2500_300_p1_20_100per/*.pdb')

In [4]:
samples_md = [md.load(x) for x in samples_unitime_2500_300_p1_20_100per]

In [5]:
data = samples_md[0][0]

In [ ]:
allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))

In [ ]:
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)

In [ ]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))

In [ ]:
from collections import Counter

In [ ]:
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])

In [ ]:
len(regspace_frames)

In [ ]:
regspace_counters = []

for x in regspace_frames:
    regspace_counters.append(Counter(x))

In [ ]:
msm_populations = np.array([ 0.00118174,  0.00377706,  0.00210663,  0.00714451,  0.01740405,
        0.00055486,  0.00384246,  0.00736303,  0.00332306,  0.00149088,
        0.00713653,  0.00396535,  0.00236742,  0.00253077,  0.08525878,
        0.14827181,  0.09523633,  0.19014794,  0.12703765,  0.28985913])

In [ ]:
shi_data = np.array([
[[],['2-1, 4-2, 6-1, 10-1, 11-2, 11-4, 11-5, 11-6, 11-7, 17-1'],['10-4, 17-4'],['15-8'],[],[],['14-4'],['13-3, 14-7']],
[[],['3-1, 3-2, 6-2, 6-3, 7-7, 10-2, 11-1, 12-3, 15-1, 17-3'],['7-1, 7-2, 11-3, 15-2, 17-2'],['15-6'],[],['14-5'],['14-2'],['13-1, 14-1']],
[[],['4-1, 7-3, 7-6, 10-3, 15-4, 15-9'],['12-1'],['14-6'],[],['13-2'],[],['9-1, 9-3']],
[['15-7'],['12-2, 19-10'],['7-5'],['16-2, 16-9'],['16-3'],['16-12, 19-9'],['14-3, 15-3, 16-7, 19-3'],['5-1, 9-2, 13-4']],
[[],['0-1, 1-1, 7-4, 13-6, 16-4, 16-5, 16-6, 19-5, 19-8, 19-12'],['1-2, 1-3, 16-1, 18-3, 18-6, 18-10, 19-2, 19-13'],['15-5, 16-11, 18-2, 18-4, 19-7, 19-11'],['18-7'],['8-1, 8-2, 8-3, 18-9, 19-6'],['13-5'],['5-2']],
[['16-10, 19-4'],['18-1, 18-8, 18-11, 19-1'],[],['16-8, 18-12'],[],['18-5'],[],[]],
[[],[],[],[],[],[],[],[]],
])         

In [14]:
shi_data2 = np.empty((7,8)).tolist()

for i in range(7):
    for j in range(8):
        if shi_data[i,j] == []:
            shi_data2[i][j] = []
        else:    
            shi_data2[i][j] = shi_data[i,j][0].split(', ')
            
shi_data2 = np.asarray(shi_data2)            

In [15]:
shi_data = shi_data2

In [16]:
shi_data_corrected = np.zeros((7,8))

In [17]:
for i in range(7):
    for j in range(8):
        data = shi_data[i,j]
        data_split = [x.split('-') for x in data]
        result = 0
        for y in data_split:
            result += msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values())))
        shi_data_corrected[i,j] = result

In [18]:
np.sum(shi_data_corrected)

0.99999999000000006

In [65]:
plt.figure()
plt.imshow(shi_data_corrected)
plt.colorbar()
plt.savefig('shi_data_corrected_2500_20.png', dpi=300)
plt.close()

In [64]:
shi_data_corrected

array([[  0.00000000e+00,   5.15008178e-02,   9.65012760e-03,
          2.96543620e-03,   0.00000000e+00,   0.00000000e+00,
          1.70517560e-03,   1.38404950e-03],
       [  0.00000000e+00,   8.84730937e-02,   1.83685043e-01,
          4.44815430e-03,   0.00000000e+00,   1.70517560e-03,
          1.10836414e-02,   3.26086720e-02],
       [  0.00000000e+00,   2.50416948e-02,   9.70642200e-04,
          8.52587800e-04,   0.00000000e+00,   1.26538500e-04,
          0.00000000e+00,   3.57811200e-04],
       [  2.96543620e-03,   4.27169490e-03,   7.36303000e-05,
          1.14283596e-02,   1.90472660e-03,   3.57354589e-02,
          1.01972260e-01,   2.06091640e-03],
       [  0.00000000e+00,   6.85623600e-02,   1.62671787e-01,
          5.14398671e-02,   1.27037650e-03,   1.87285547e-02,
          2.78384700e-04,   1.33166400e-04],
       [  3.85095460e-03,   1.01486580e-01,   0.00000000e+00,
          6.98455630e-03,   0.00000000e+00,   7.62225900e-03,
          0.00000000e+00,   0.0

In [21]:
# plot this as an MSM figure, but first do the imshow with blue-white and red-white color scheme

In [22]:
plt.figure()
plt.imshow(shi_data_corrected, cmap='Blues')
plt.colorbar()
plt.savefig('shi_data_corrected_2500_20_blue.png', dpi=300)
plt.close()

In [23]:
plt.figure()
plt.imshow(shi_data_corrected, cmap='Reds')
plt.colorbar()
plt.savefig('shi_data_corrected_2500_20_red.png', dpi=300)
plt.close()

In [19]:
plt.figure()
plt.imshow(np.log10(shi_data_corrected), cmap='Blues')
plt.colorbar()
plt.savefig('shi_data_corrected_2500_20_blue_log10.png', dpi=300)
plt.close()

In [20]:
plt.figure()
plt.imshow(np.log10(shi_data_corrected), cmap='Reds')
plt.colorbar()
plt.savefig('shi_data_corrected_2500_20_red_log10.png', dpi=300)
plt.close()

In [46]:
# ok plot as an MSM figure in pyemma

In [67]:
P = np.zeros((56,56))

pos = np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], 
[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7], 
[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], 
[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],
[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7],
[5,0], [5,1], [5,2], [5,3], [5,4], [5,5], [5,6], [5,7],
[6,0], [6,1], [6,2], [6,3], [6,4], [6,5], [6,6], [6,7]])

state_labels = []
for i in range(56):
    state_labels.append('')

In [62]:
plt.figure()
pyemma.plots.plot_network(P, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels)
plt.savefig('shi_data_corrected_2500_msm.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [66]:
P.shape

(56, 56)

In [68]:
P[0,2] = 1
P[50,7] = 1
P[30,6] = 1
P[2,40] = 1
P[20,4] = 1
P[0,8] = 1
P[6,25] = 1
P[12,15] = 1
P[6,0] = 1
P[15,33] = 1
P[22,44] = 1
P[11,17] = 1
P[22,45] = 1
P[1,3] = 1

In [69]:
plt.figure()
pyemma.plots.plot_network(P, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None)
plt.savefig('shi_data_corrected_2500_msm_arrows.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [70]:
# ok that was a test of arrows - now let's calculate actual connectivities

In [71]:
P_spy = np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         1.,  1.,  1.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  1.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         1.,  1.,  1.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  1.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  0.,  0.,
         0.,  1.,  1.,  1.,  1.,  1.,  1.],
       [ 0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.]])

In [74]:
shi_data

array([[[],
        ['2-1', '4-2', '6-1', '10-1', '11-2', '11-4', '11-5', '11-6', '11-7', '17-1'],
        ['10-4', '17-4'], ['15-8'], [], [], ['14-4'], ['13-3', '14-7']],
       [[],
        ['3-1', '3-2', '6-2', '6-3', '7-7', '10-2', '11-1', '12-3', '15-1', '17-3'],
        ['7-1', '7-2', '11-3', '15-2', '17-2'], ['15-6'], [], ['14-5'],
        ['14-2'], ['13-1', '14-1']],
       [[], ['4-1', '7-3', '7-6', '10-3', '15-4', '15-9'], ['12-1'],
        ['14-6'], [], ['13-2'], [], ['9-1', '9-3']],
       [['15-7'], ['12-2', '19-10'], ['7-5'], ['16-2', '16-9'], ['16-3'],
        ['16-12', '19-9'], ['14-3', '15-3', '16-7', '19-3'],
        ['5-1', '9-2', '13-4']],
       [[],
        ['0-1', '1-1', '7-4', '13-6', '16-4', '16-5', '16-6', '19-5', '19-8', '19-12'],
        ['1-2', '1-3', '16-1', '18-3', '18-6', '18-10', '19-2', '19-13'],
        ['15-5', '16-11', '18-2', '18-4', '19-7', '19-11'], ['18-7'],
        ['8-1', '8-2', '8-3', '18-9', '19-6'], ['13-5'], ['5-2']],
       [['16-10', '19

In [75]:
shi_data.shape

(7, 8)

In [85]:
shi_data_macrostate_numbers = []
shi_data_macrostate_numbers_sets = []

for x in shi_data:
    shi_data_macrostate_numbers.append([])
    shi_data_macrostate_numbers_sets.append([])
    for y in x:
        shi_data_macrostate_numbers[-1].append([])
        for z in y:
            shi_data_macrostate_numbers[-1][-1].append(z.split('-')[0])
        shi_data_macrostate_numbers_sets[-1].append(list(set(shi_data_macrostate_numbers[-1][-1])))

In [87]:
shi_data_macrostate_numbers_sets

[[[],
  ['6', '2', '4', '17', '10', '11'],
  ['10', '17'],
  ['15'],
  [],
  [],
  ['14'],
  ['13', '14']],
 [[],
  ['6', '15', '3', '7', '17', '10', '12', '11'],
  ['11', '15', '17', '7'],
  ['15'],
  [],
  ['14'],
  ['14'],
  ['13', '14']],
 [[], ['15', '10', '4', '7'], ['12'], ['14'], [], ['13'], [], ['9']],
 [['15'],
  ['19', '12'],
  ['7'],
  ['16'],
  ['16'],
  ['19', '16'],
  ['15', '14', '19', '16'],
  ['5', '9', '13']],
 [[],
  ['0', '7', '13', '1', '16', '19'],
  ['19', '18', '1', '16'],
  ['15', '18', '19', '16'],
  ['18'],
  ['19', '8', '18'],
  ['13'],
  ['5']],
 [['19', '16'], ['19', '18'], [], ['18', '16'], [], ['18'], [], []],
 [[], [], [], [], [], [], [], []]]

In [96]:
P = np.zeros((56,56))

pos = np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], 
[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7], 
[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], 
[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],
[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7],
[5,0], [5,1], [5,2], [5,3], [5,4], [5,5], [5,6], [5,7],
[6,0], [6,1], [6,2], [6,3], [6,4], [6,5], [6,6], [6,7]])

state_labels = []
for i in range(56):
    state_labels.append('')

In [107]:
shi_data_macrostate_numbers_sets_flat = np.array(shi_data_macrostate_numbers_sets).flatten()

In [109]:
strmacrostate_macrostate_dict = dict()

for i in range(len(shi_data_macrostate_numbers_sets_flat)):
    for x in shi_data_macrostate_numbers_sets_flat[i]:
        if x not in strmacrostate_macrostate_dict:
            strmacrostate_macrostate_dict[x] = [i]
        else:
            strmacrostate_macrostate_dict[x].append(i)

In [110]:
strmacrostate_macrostate_dict

{'0': [33],
 '1': [33, 34],
 '10': [1, 2, 9, 17],
 '11': [1, 9, 10],
 '12': [9, 18, 25],
 '13': [7, 15, 21, 31, 33, 38],
 '14': [6, 7, 13, 14, 15, 19, 30],
 '15': [3, 9, 10, 11, 17, 24, 30, 35],
 '16': [27, 28, 29, 30, 33, 34, 35, 40, 43],
 '17': [1, 2, 9, 10],
 '18': [34, 35, 36, 37, 41, 43, 45],
 '19': [25, 29, 30, 33, 34, 35, 37, 40, 41],
 '2': [1],
 '3': [9],
 '4': [1, 17],
 '5': [31, 39],
 '6': [1, 9],
 '7': [9, 10, 17, 26, 33],
 '8': [37],
 '9': [23, 31]}

In [112]:
P.shape

(56, 56)

In [118]:
P = np.zeros((56,56))

pos = np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], 
[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7], 
[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], 
[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],
[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7],
[5,0], [5,1], [5,2], [5,3], [5,4], [5,5], [5,6], [5,7],
[6,0], [6,1], [6,2], [6,3], [6,4], [6,5], [6,6], [6,7]])

state_labels = []
for i in range(56):
    state_labels.append('')

In [121]:
P = np.zeros((56,56))

for strmacrostate in range(56):
    for macrostate in strmacrostate_macrostate_dict:
        if strmacrostate in strmacrostate_macrostate_dict[macrostate]:
            for other_strmacrostate in strmacrostate_macrostate_dict[macrostate]:
                P[strmacrostate, other_strmacrostate] = 1
                #P[other_strmacrostate, strmacrostate] = 1

In [135]:
plt.figure()
pyemma.plots.plot_network(P, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_only_same_macrostate.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [125]:
P_ = np.zeros((56,56))

for x in range(56):
    for y in range(x+1):
        P_[x,y] = P[x,y]

In [133]:
plt.figure()
pyemma.plots.plot_network(P_, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_only_same_macrostate_half.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [136]:
# ok go further - now add connectivities between macrostates

In [156]:
P = np.zeros((56,56))

pos = np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], 
[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7], 
[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], 
[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],
[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7],
[5,0], [5,1], [5,2], [5,3], [5,4], [5,5], [5,6], [5,7],
[6,0], [6,1], [6,2], [6,3], [6,4], [6,5], [6,6], [6,7]])

state_labels = []
for i in range(56):
    state_labels.append('')

In [157]:
# only within macrostate 
#for strmacrostate in range(56):
#    for macrostate in strmacrostate_macrostate_dict:
#        if strmacrostate in strmacrostate_macrostate_dict[macrostate]:
#            for other_strmacrostate in strmacrostate_macrostate_dict[macrostate]:
#                P[strmacrostate, other_strmacrostate] = 1

In [158]:
# between macrostates

In [159]:
for strmacrostate in range(56):
    for macrostate in strmacrostate_macrostate_dict:
        if strmacrostate in strmacrostate_macrostate_dict[macrostate]:
            # then check what other macrostates this macrostate is connected to in P_spy, then add strmacrostate connections
            # between this strmacrostate and all others in all those macrostates
            connectivity = P_spy[int(macrostate)]
            for i in range(20):
                if connectivity[i] == 1:
                    macrostate2 = strmacrostate_macrostate_dict[str(i)]
                    for other_strmacrostate in macrostate2:
                        P[strmacrostate, other_strmacrostate] = 1

In [160]:
plt.figure()
pyemma.plots.plot_network(P, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_all.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [161]:
P_ = np.zeros((56,56))

for x in range(56):
    for y in range(x+1):
        P_[x,y] = P[x,y]

In [162]:
plt.figure()
pyemma.plots.plot_network(P_, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_all_half.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [163]:
# well it's pretty messy - let's try just between macrostates, without within macrostate

In [164]:
P = np.zeros((56,56))

pos = np.array([[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7], 
[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7], 
[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], 
[3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],
[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7],
[5,0], [5,1], [5,2], [5,3], [5,4], [5,5], [5,6], [5,7],
[6,0], [6,1], [6,2], [6,3], [6,4], [6,5], [6,6], [6,7]])

state_labels = []
for i in range(56):
    state_labels.append('')

In [165]:
for strmacrostate in range(56):
    for macrostate in strmacrostate_macrostate_dict:
        if strmacrostate in strmacrostate_macrostate_dict[macrostate]:
            # then check what other macrostates this macrostate is connected to in P_spy, then add strmacrostate connections
            # between this strmacrostate and all others in all those macrostates
            connectivity = P_spy[int(macrostate)]
            for i in range(20):
                if connectivity[i] == 1 and i != int(macrostate):
                    macrostate2 = strmacrostate_macrostate_dict[str(i)]
                    for other_strmacrostate in macrostate2:
                        P[strmacrostate, other_strmacrostate] = 1

In [168]:
plt.figure()
pyemma.plots.plot_network(P, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_only_betweenmacro.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [167]:
P_ = np.zeros((56,56))

for x in range(56):
    for y in range(x+1):
        P_[x,y] = P[x,y]

In [169]:
plt.figure()
pyemma.plots.plot_network(P_, pos=pos, state_sizes=shi_data_corrected.flatten(), state_labels=state_labels, arrow_labels=None, arrow_scale=0.1)
plt.savefig('shi_data_corrected_2500_msm_arrows_only_betweenmacro_half.png', dpi=300)
plt.close()

/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/Users/rafalpwiewiora/anaconda3/lib/python3.5/site-packages/pyemma/plots/networks.py:70: UserWarning: The layout optimization method will take a long time for large networks! It is recommended to coarse grain your model first!
  warnings.warn("The layout optimization method will take a long"


In [45]:
# runs of origin

In [46]:
from collections import Counter

glob_output = np.load('../glob_output.npy')
glob_output = [x.decode('UTF-8') for x in glob_output]

samples = np.load('../macro_sample/unitime_2500_300_p1_20_100per/samples_300_p1_20_100per.npy')

run_results = []

for macrostate in samples:
    macrostate_results = []
    for frame in macrostate:
        x = glob_output[frame[0]]
        y = int(x.split('-')[0][9:])
        if x[:5] == '11707':
            if y <= 19:
                macrostate_results.append('1zkk')
            elif y in [20,21] or (y >= 42 and y <= 59):
                macrostate_results.append('4ij8')
            elif y in [22,25,26,27,28,31,32,33,34,35] or (y >= 60 and y <= 69):
                macrostate_results.append('Apo')
            elif y in [23,24,29,30,36,37,38,39,40,41] or y >= 70:
                macrostate_results.append('Inhibitor')
        elif x[:5] == '11709':
            if y in [0,2]:
                macrostate_results.append('1zkk_apo')
            elif y in [1,3]:
                macrostate_results.append('4ij8_apo')
    
    run_results.append(macrostate_results)
    
run_results_counters = []

for x in run_results:
    run_results_counters.append(Counter(x))    

In [47]:
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))
    
regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])
        
regspace_5_by_cluster_runs_of_origin = []

for i in range(len(regspace_frames)):
    clusters = regspace_frames[i]
    runs = run_results[i]
    
    regspace_5_by_cluster_runs_of_origin.append(dict())
    for j in range(len(clusters)):
        if clusters[j] not in regspace_5_by_cluster_runs_of_origin[-1]:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]] = [runs[j]]
        else:
            regspace_5_by_cluster_runs_of_origin[-1][clusters[j]].append(runs[j])
            
regspace_5_by_cluster_runs_of_origin_counters = []

for macrostate in regspace_5_by_cluster_runs_of_origin:
    regspace_5_by_cluster_runs_of_origin_counters.append(dict())
    for cluster in macrostate:
        regspace_5_by_cluster_runs_of_origin_counters[-1][cluster] = Counter(macrostate[cluster])            

In [48]:
shi_data_runs_origin = [
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
[dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()],
]

for i in range(7):
    for j in range(8):
        data = shi_data[i,j]
        data_split = [x.split('-') for x in data]
        output_dict = shi_data_runs_origin[i][j]
        for y in data_split:
            weight = msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values())))
            weight = weight / np.sum([msm_populations[int(y[0])] * (regspace_counters[int(y[0])][int(y[1])-1] / np.sum(list(regspace_counters[int(y[0])].values()))) for y in data_split])
            counter = regspace_5_by_cluster_runs_of_origin_counters[int(y[0])][int(y[1])-1]
            temp_dict = dict()
            for run_of_origin in counter:
                temp_dict[run_of_origin] = counter[run_of_origin] / np.sum([counter[x] for x in counter])
            
            for x in temp_dict:
                temp_dict[x] = temp_dict[x] * weight
                if x not in output_dict:
                    output_dict[x] = temp_dict[x]
                else:
                    output_dict[x] += temp_dict[x]

In [49]:
shi_data_runs_origin

[[{},
  {'1zkk': 0.11264689470620406,
   '1zkk_apo': 0.29873863867070477,
   '4ij8': 0.042614033985301107,
   '4ij8_apo': 0.14768537520194486,
   'Apo': 0.2464557543394971,
   'Inhibitor': 0.1518593030963481},
  {'1zkk': 0.40887432410738278,
   '4ij8_apo': 0.39408378392841148,
   'Apo': 0.19704189196420574},
  {'4ij8_apo': 1.0},
  {},
  {},
  {'1zkk_apo': 0.5, '4ij8': 0.5},
  {'1zkk': 0.10971153849627485,
   '1zkk_apo': 0.23770833340859554,
   '4ij8_apo': 0.018285256416045809,
   'Apo': 0.018285256416045809,
   'Inhibitor': 0.61600961526303799}],
 [{},
  {'1zkk': 0.17909477602002288,
   '1zkk_apo': 0.047424668049106548,
   '4ij8': 0.086884638917063203,
   '4ij8_apo': 0.10907595514544553,
   'Apo': 0.24806252705956863,
   'Inhibitor': 0.32945743480879319},
  {'1zkk': 0.15962739509457763,
   '1zkk_apo': 0.10351846677414935,
   '4ij8': 0.11405599596267182,
   '4ij8_apo': 0.15947425143317112,
   'Apo': 0.22441910496154999,
   'Inhibitor': 0.23890478577388014},
  {'Apo': 0.33333333333333331

In [1]:
# now make MSM PIECHARTS - i.e. we're gonna take all 20 macrostates and plot them as pie-charts of 'submacrostate'
# populations 

In [ ]:
# for 2500 - model 5 first

In [39]:
import numpy as np
import glob
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pyemma
import msmbuilder
from msmbuilder.msm import MarkovStateModel
from msmbuilder.lumping import PCCAPlus

import mdtraj as md

import glob
from msmbuilder.cluster import RegularSpatial

In [40]:
samples_unitime_2500_300_p1_20_100per = glob.glob('../macro_sample/unitime_2500_300_p1_20_100per/*.pdb')

samples_md = [md.load(x) for x in samples_unitime_2500_300_p1_20_100per]
data = samples_md[0][0]

allatoms_data_selection = data.top.select('not element H and resid > 24')
samples_md_noNflank_allatoms = []

for x in samples_md:
    samples_md_noNflank_allatoms.append(x.atom_slice(allatoms_data_selection))
    
regspace_objects_noNflank_allatoms_5 = []

for x in samples_md_noNflank_allatoms:
    regspace_objects_noNflank_allatoms_5.append(RegularSpatial(0.5, metric='rmsd'))
    regspace_objects_noNflank_allatoms_5[-1].fit(x)
    
regspace_dtrajs = []

for i in range(len(samples_md_noNflank_allatoms)):
    samples = samples_md_noNflank_allatoms[i]
    regspace_dtrajs.append(regspace_objects_noNflank_allatoms_5[i].transform(samples))
    
from collections import Counter

regspace_frames = []

for x in regspace_dtrajs:
    regspace_frames.append([])
    for y in x:
        regspace_frames[-1].append(y[0])
        
regspace_counters = []

for x in regspace_frames:
    regspace_counters.append(Counter(x))        

In [41]:
msm_populations = np.array([ 0.00118174,  0.00377706,  0.00210663,  0.00714451,  0.01740405,
        0.00055486,  0.00384246,  0.00736303,  0.00332306,  0.00149088,
        0.00713653,  0.00396535,  0.00236742,  0.00253077,  0.08525878,
        0.14827181,  0.09523633,  0.19014794,  0.12703765,  0.28985913])

In [42]:
shi_data = np.array([
[[],['2-1, 4-2, 6-1, 10-1, 11-2, 11-4, 11-5, 11-6, 11-7, 17-1'],['10-4, 17-4'],['15-8'],[],[],['14-4'],['13-3, 14-7']],
[[],['3-1, 3-2, 6-2, 6-3, 7-7, 10-2, 11-1, 12-3, 15-1, 17-3'],['7-1, 7-2, 11-3, 15-2, 17-2'],['15-6'],[],['14-5'],['14-2'],['13-1, 14-1']],
[[],['4-1, 7-3, 7-6, 10-3, 15-4, 15-9'],['12-1'],['14-6'],[],['13-2'],[],['9-1, 9-3']],
[['15-7'],['12-2, 19-10'],['7-5'],['16-2, 16-9'],['16-3'],['16-12, 19-9'],['14-3, 15-3, 16-7, 19-3'],['5-1, 9-2, 13-4']],
[[],['0-1, 1-1, 7-4, 13-6, 16-4, 16-5, 16-6, 19-5, 19-8, 19-12'],['1-2, 1-3, 16-1, 18-3, 18-6, 18-10, 19-2, 19-13'],['15-5, 16-11, 18-2, 18-4, 19-7, 19-11'],['18-7'],['8-1, 8-2, 8-3, 18-9, 19-6'],['13-5'],['5-2']],
[['16-10, 19-4'],['18-1, 18-8, 18-11, 19-1'],[],['16-8, 18-12'],[],['18-5'],[],[]],
[[],[],[],[],[],[],[],[]],        
])         

In [5]:
# first make all piecharts the same size, just put the sub macrostate info in, 
# then try making them a size proportional to the MSM populations

In [6]:
# so we have regspace_counters to get submacrostate populations from, and shi_data - so need a list of 20 dictionaries, 
# where dictionaries will be key - submacrostate indexes, value - submacrostate population within the macrostate

In [7]:
regspace_counters

[Counter({0: 100}),
 Counter({0: 79, 1: 19, 2: 2}),
 Counter({0: 100}),
 Counter({0: 98, 1: 2}),
 Counter({0: 98, 1: 2}),
 Counter({0: 76, 1: 24}),
 Counter({0: 54, 1: 44, 2: 2}),
 Counter({0: 29, 1: 28, 2: 13, 3: 11, 4: 1, 5: 12, 6: 6}),
 Counter({0: 59, 1: 33, 2: 8}),
 Counter({0: 13, 1: 76, 2: 11}),
 Counter({0: 42, 1: 53, 2: 3, 3: 2}),
 Counter({0: 16, 1: 26, 2: 30, 3: 5, 4: 13, 5: 1, 6: 9}),
 Counter({0: 41, 1: 58, 2: 1}),
 Counter({0: 42, 1: 5, 2: 21, 3: 20, 4: 11, 5: 1}),
 Counter({0: 37, 1: 13, 2: 44, 3: 2, 4: 2, 5: 1, 6: 1}),
 Counter({0: 26, 1: 51, 2: 7, 3: 1, 4: 5, 5: 3, 6: 2, 7: 2, 8: 3}),
 Counter({0: 54,
          1: 10,
          2: 2,
          3: 3,
          4: 4,
          5: 8,
          6: 2,
          7: 6,
          8: 2,
          9: 1,
          10: 7,
          11: 1}),
 Counter({0: 22, 1: 54, 2: 19, 3: 5}),
 Counter({0: 10,
          1: 12,
          2: 21,
          3: 6,
          4: 6,
          5: 13,
          6: 1,
          7: 3,
          8: 3,
      

In [8]:
# actually regspace_counters is all we need already!!

In [9]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/regspace_analysis'

In [10]:
cd piecharts

/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/regspace_analysis/piecharts


In [33]:
# this doesn't plot a from-the-top view
plt.figure()
plt.pie(list(regspace_counters[1].values()), labels=[x+1 for x in range(len(regspace_counters[1]))], autopct='%1.1f%%')
plt.savefig('test.png', dpi=300)
plt.close()

In [35]:
# this plots a from-the-top view - i.e. the chart is a circle
fig1, ax1 = plt.subplots()
ax1.pie(list(regspace_counters[1].values()), labels=[x+1 for x in range(len(regspace_counters[1]))], autopct='%1.1f%%')
ax1.axis('equal')
plt.savefig('test.png', dpi=300)
plt.close()

In [36]:
# ok now plot them for all 20 macrostates and put in one pdf for now - then can try to put them on one page

In [40]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("test.pdf")

for x in regspace_counters:
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=[x+1 for x in range(len(x))], autopct='%1.1f%%')
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [41]:
# ok so just realized regspace_counters is not all we need already - we need not populations of the sub-macrostates, 
# but populations of 'structural-macrostates' in macrostates

In [42]:
# so let's take regspace_counters and make a new dict - change the submacrostate labels to structural macrostate labels

In [44]:
shi_data.shape

(7, 8)

In [45]:
7*8

56

In [51]:
shi_data[0,1]

['2-1, 4-2, 6-1, 10-1, 11-2, 11-4, 11-5, 11-6, 11-7, 17-1']

In [43]:
shi_data2 = np.empty((7,8)).tolist()

for i in range(7):
    for j in range(8):
        if shi_data[i,j] == []:
            shi_data2[i][j] = []
        else:    
            shi_data2[i][j] = shi_data[i,j][0].split(', ')
            
shi_data2 = np.asarray(shi_data2)            

In [44]:
shi_data = shi_data2

In [13]:
shi_data

array([[[],
        ['2-1', '4-2', '6-1', '10-1', '11-2', '11-4', '11-5', '11-6', '11-7', '17-1'],
        ['10-4', '17-4'], ['15-8'], [], [], ['14-4'], ['13-3', '14-7']],
       [[],
        ['3-1', '3-2', '6-2', '6-3', '7-7', '10-2', '11-1', '12-3', '15-1', '17-3'],
        ['7-1', '7-2', '11-3', '15-2', '17-2'], ['15-6'], [], ['14-5'],
        ['14-2'], ['13-1', '14-1']],
       [[], ['4-1', '7-3', '7-6', '10-3', '15-4', '15-9'], ['12-1'],
        ['14-6'], [], ['13-2'], [], ['9-1', '9-3']],
       [['15-7'], ['12-2', '19-10'], ['7-5'], ['16-2', '16-9'], ['16-3'],
        ['16-12', '19-9'], ['14-3', '15-3', '16-7', '19-3'],
        ['5-1', '9-2', '13-4']],
       [[],
        ['0-1', '1-1', '7-4', '13-6', '16-4', '16-5', '16-6', '19-5', '19-8', '19-12'],
        ['1-2', '1-3', '16-1', '18-3', '18-6', '18-10', '19-2', '19-13'],
        ['15-5', '16-11', '18-2', '18-4', '19-7', '19-11'], ['18-7'],
        ['8-1', '8-2', '8-3', '18-9', '19-6'], ['13-5'], ['5-2']],
       [['16-10', '19

In [56]:
'2-1' in shi_data[0,1]

True

In [57]:
regspace_counters

[Counter({0: 100}),
 Counter({0: 79, 1: 19, 2: 2}),
 Counter({0: 100}),
 Counter({0: 98, 1: 2}),
 Counter({0: 98, 1: 2}),
 Counter({0: 76, 1: 24}),
 Counter({0: 54, 1: 44, 2: 2}),
 Counter({0: 29, 1: 28, 2: 13, 3: 11, 4: 1, 5: 12, 6: 6}),
 Counter({0: 59, 1: 33, 2: 8}),
 Counter({0: 13, 1: 76, 2: 11}),
 Counter({0: 42, 1: 53, 2: 3, 3: 2}),
 Counter({0: 16, 1: 26, 2: 30, 3: 5, 4: 13, 5: 1, 6: 9}),
 Counter({0: 41, 1: 58, 2: 1}),
 Counter({0: 42, 1: 5, 2: 21, 3: 20, 4: 11, 5: 1}),
 Counter({0: 37, 1: 13, 2: 44, 3: 2, 4: 2, 5: 1, 6: 1}),
 Counter({0: 26, 1: 51, 2: 7, 3: 1, 4: 5, 5: 3, 6: 2, 7: 2, 8: 3}),
 Counter({0: 54,
          1: 10,
          2: 2,
          3: 3,
          4: 4,
          5: 8,
          6: 2,
          7: 6,
          8: 2,
          9: 1,
          10: 7,
          11: 1}),
 Counter({0: 22, 1: 54, 2: 19, 3: 5}),
 Counter({0: 10,
          1: 12,
          2: 21,
          3: 6,
          4: 6,
          5: 13,
          6: 1,
          7: 3,
          8: 3,
      

In [45]:
shi_data_flatten = shi_data.flatten()

In [46]:
# combined code here
shi_data_flatten = shi_data.flatten()

regspace_counters_struct_macro = []

for x in range(len(regspace_counters)):
    new_dict = dict()
    macrostate = regspace_counters[x]
    for y in macrostate.keys():
        for i in range(len(shi_data_flatten)):
            data = shi_data_flatten[i]
            if '%s-%s' % (x,y+1) in data:
                new_key = i
                break
        if not new_key in new_dict:        
            new_dict[new_key] = macrostate[y]
        else:
            new_dict[new_key] += macrostate[y]
    regspace_counters_struct_macro.append(new_dict)    
    
regspace_counters_struct_macro = []

for x in range(len(regspace_counters)):
    new_dict = dict()
    macrostate = regspace_counters[x]
    for y in macrostate.keys():
        for i in range(len(shi_data_flatten)):
            data = shi_data_flatten[i]
            if '%s-%s' % (x,y+1) in data:
                new_key = i
                break
        if not new_key in new_dict:        
            new_dict[new_key] = macrostate[y]
        else:
            new_dict[new_key] += macrostate[y]
    regspace_counters_struct_macro.append(new_dict)        

In [15]:
shi_data_flatten

array([[],
       ['2-1', '4-2', '6-1', '10-1', '11-2', '11-4', '11-5', '11-6', '11-7', '17-1'],
       ['10-4', '17-4'], ['15-8'], [], [], ['14-4'], ['13-3', '14-7'], [],
       ['3-1', '3-2', '6-2', '6-3', '7-7', '10-2', '11-1', '12-3', '15-1', '17-3'],
       ['7-1', '7-2', '11-3', '15-2', '17-2'], ['15-6'], [], ['14-5'],
       ['14-2'], ['13-1', '14-1'], [],
       ['4-1', '7-3', '7-6', '10-3', '15-4', '15-9'], ['12-1'], ['14-6'],
       [], ['13-2'], [], ['9-1', '9-3'], ['15-7'], ['12-2', '19-10'],
       ['7-5'], ['16-2', '16-9'], ['16-3'], ['16-12', '19-9'],
       ['14-3', '15-3', '16-7', '19-3'], ['5-1', '9-2', '13-4'], [],
       ['0-1', '1-1', '7-4', '13-6', '16-4', '16-5', '16-6', '19-5', '19-8', '19-12'],
       ['1-2', '1-3', '16-1', '18-3', '18-6', '18-10', '19-2', '19-13'],
       ['15-5', '16-11', '18-2', '18-4', '19-7', '19-11'], ['18-7'],
       ['8-1', '8-2', '8-3', '18-9', '19-6'], ['13-5'], ['5-2'],
       ['16-10', '19-4'], ['18-1', '18-8', '18-11', '19-1'], [],

In [9]:
regspace_counters_struct_macro = []

for x in range(len(regspace_counters)):
    new_dict = dict()
    macrostate = regspace_counters[x]
    for y in macrostate.keys():
        for i in range(len(shi_data_flatten)):
            data = shi_data_flatten[i]
            if '%s-%s' % (x,y+1) in data:
                new_key = i
                break
        if not new_key in new_dict:        
            new_dict[new_key] = macrostate[y]
        else:
            new_dict[new_key] += macrostate[y]
    regspace_counters_struct_macro.append(new_dict)    

In [17]:
regspace_counters_struct_macro

[{33: 100},
 {33: 79, 34: 21},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 76, 39: 24},
 {1: 54, 9: 46},
 {9: 6, 10: 57, 17: 25, 26: 1, 33: 11},
 {37: 100},
 {23: 24, 31: 76},
 {1: 42, 2: 2, 9: 53, 17: 3},
 {1: 54, 9: 16, 10: 30},
 {9: 1, 18: 41, 25: 58},
 {7: 21, 15: 42, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {3: 2, 9: 26, 10: 51, 11: 3, 17: 4, 24: 2, 30: 7, 35: 5},
 {27: 12, 28: 2, 29: 1, 30: 2, 33: 15, 34: 54, 35: 7, 40: 1, 43: 6},
 {1: 22, 2: 5, 9: 19, 10: 54},
 {34: 55, 35: 18, 36: 1, 37: 3, 41: 16, 43: 1, 45: 6},
 {25: 1, 29: 12, 30: 18, 33: 17, 34: 14, 35: 5, 37: 4, 40: 1, 41: 28}]

In [21]:
for i in range(20):
    if not len(regspace_counters[i]) == len(regspace_counters_struct_macro[i]):
        print(i)

1
3
6
7
8
9
11
15
16
18
19


In [85]:
regspace_counters[1]

Counter({0: 79, 1: 19, 2: 2})

In [87]:
shi_data_flatten[34]

['1-2', '1-3', '16-1', '18-3', '18-6', '18-10', '19-2', '19-13']

In [88]:
# ok looks good! plot

In [22]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("model5_piecharts.pdf")

for x in regspace_counters_struct_macro:
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%')
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [ ]:
# now do the 'if in the nearest 8 - count the same' - i.e. we're gonna take the most populated submacrostate in 
# a macrostate, and see if the macrostate also has any of the 8 closest submacrostates on the heatmap - if so 
# call those the biggest one too and add their populations to it - replot like that - put on the MSM network

In [23]:
regspace_counters_struct_macro

[{33: 100},
 {33: 79, 34: 21},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 76, 39: 24},
 {1: 54, 9: 46},
 {9: 6, 10: 57, 17: 25, 26: 1, 33: 11},
 {37: 100},
 {23: 24, 31: 76},
 {1: 42, 2: 2, 9: 53, 17: 3},
 {1: 54, 9: 16, 10: 30},
 {9: 1, 18: 41, 25: 58},
 {7: 21, 15: 42, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {3: 2, 9: 26, 10: 51, 11: 3, 17: 4, 24: 2, 30: 7, 35: 5},
 {27: 12, 28: 2, 29: 1, 30: 2, 33: 15, 34: 54, 35: 7, 40: 1, 43: 6},
 {1: 22, 2: 5, 9: 19, 10: 54},
 {34: 55, 35: 18, 36: 1, 37: 3, 41: 16, 43: 1, 45: 6},
 {25: 1, 29: 12, 30: 18, 33: 17, 34: 14, 35: 5, 37: 4, 40: 1, 41: 28}]

In [24]:
regspace_counters_struct_macro2 = []

In [ ]:
# make the heatmap shaped map of the 0-55 indexes so I can implement rules for being in the closest 8

In [18]:
submacro_array = np.array([
[0,1,2,3,4,5,6,7],
[8,9,10,11,12,13,14,15],
[16,17,18,19,20,21,22,23],
[24,25,26,27,28,29,30,31],
[32,33,34,35,36,37,38,39],
[40,41,42,43,44,45,46,47],
[48,49,50,51,52,53,54,55],
])

In [30]:
submacro_array.shape

(7, 8)

In [31]:
submacro_array

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55]])

In [19]:
regspace_counters_struct_macro2 = []

for x in regspace_counters_struct_macro:
    new_dict = dict()
    absorb_list = []
    for key in x:
        value = x[key]
        if value == np.max(list(x.values())): # then this is the biggest one
            absorb_list = [key]
            index0 = int(np.where(submacro_array == key)[0])
            index1 = int(np.where(submacro_array == key)[1])
            indices_to_check = [(index0-1, index1), (index0-1, index1-1), (index0-1, index1+1), (index0, index1-1), (index0, index1+1), (index0+1, index1), (index0+1, index1+1), (index0+1, index1-1)]
            indices_to_check = [x for x in indices_to_check if x[0] >= 0 and x[0] <= 6 and x[1] >= 0 and x[1] <= 7]
            submacro_to_check = [submacro_array[x[0], x[1]] for x in indices_to_check]
            for submacro in submacro_to_check:
                if submacro in x.keys():
                    absorb_list.append(submacro)
    # ok so now we have empty absorb_list if x was empty, and one element absorb_list if it wasn't
    # more than one element there - we gotta absorb some submacrostates into the biggest one
    if len(absorb_list) > 1:
        new_dict[absorb_list[0]] = x[absorb_list[0]]
        for y in absorb_list[1:]:
            new_dict[absorb_list[0]] += x[y]
        for key in x:
            if key not in absorb_list:
                new_dict[key] = x[key]
        regspace_counters_struct_macro2.append(new_dict)
    else:
        regspace_counters_struct_macro2.append(x)            

In [20]:
regspace_counters_struct_macro2

[{33: 100},
 {33: 100},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 100},
 {1: 100},
 {10: 88, 26: 1, 33: 11},
 {37: 100},
 {31: 100},
 {9: 100},
 {1: 100},
 {9: 1, 25: 99},
 {15: 63, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {10: 86, 24: 2, 30: 7, 35: 5},
 {28: 2, 29: 1, 30: 2, 34: 94, 40: 1},
 {10: 100},
 {34: 90, 36: 1, 37: 3, 45: 6},
 {25: 1, 29: 12, 30: 18, 35: 5, 37: 4, 41: 60}]

In [21]:
regspace_counters_struct_macro

[{33: 100},
 {33: 79, 34: 21},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 76, 39: 24},
 {1: 54, 9: 46},
 {9: 6, 10: 57, 17: 25, 26: 1, 33: 11},
 {37: 100},
 {23: 24, 31: 76},
 {1: 42, 2: 2, 9: 53, 17: 3},
 {1: 54, 9: 16, 10: 30},
 {9: 1, 18: 41, 25: 58},
 {7: 21, 15: 42, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {3: 2, 9: 26, 10: 51, 11: 3, 17: 4, 24: 2, 30: 7, 35: 5},
 {27: 12, 28: 2, 29: 1, 30: 2, 33: 15, 34: 54, 35: 7, 40: 1, 43: 6},
 {1: 22, 2: 5, 9: 19, 10: 54},
 {34: 55, 35: 18, 36: 1, 37: 3, 41: 16, 43: 1, 45: 6},
 {25: 1, 29: 12, 30: 18, 33: 17, 34: 14, 35: 5, 37: 4, 40: 1, 41: 28}]

In [50]:
# ok plot version 2!!

In [51]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("model5_piecharts_absorbed.pdf")

for x in regspace_counters_struct_macro2:
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%')
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [52]:
pwd

'/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/regspace_analysis/piecharts'

In [54]:
# last improvement - submacrostates should have unique colors - let's see how many we need and get a list of 
# a bit more, in case we need more for other models

In [22]:
regspace_counters_struct_macro2

[{33: 100},
 {33: 100},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 100},
 {1: 100},
 {10: 88, 26: 1, 33: 11},
 {37: 100},
 {31: 100},
 {9: 100},
 {1: 100},
 {9: 1, 25: 99},
 {15: 63, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {10: 86, 24: 2, 30: 7, 35: 5},
 {28: 2, 29: 1, 30: 2, 34: 94, 40: 1},
 {10: 100},
 {34: 90, 36: 1, 37: 3, 45: 6},
 {25: 1, 29: 12, 30: 18, 35: 5, 37: 4, 41: 60}]

In [25]:
list_of_keys = []

for x in regspace_counters_struct_macro2:
    for y in x:
        list_of_keys.append(y)

In [27]:
set(list_of_keys)

{1,
 6,
 7,
 9,
 10,
 13,
 14,
 15,
 17,
 19,
 21,
 24,
 25,
 26,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 45}

In [28]:
len(set(list_of_keys))

27

In [29]:
# so 20 macrostates classified with 27 'structural macrostates' - I think that's pretty alright?

In [32]:
# let's find a palette of 50 colors if possible, just use 27 first ones here - actually do 35 colors
# from iWantHue

In [33]:
colors = ["#d83f9a",
"#7edf48",
"#5b37c4",
"#d8d933",
"#c449dc",
"#58c35f",
"#88349b",
"#cfdb6d",
"#686edb",
"#82a038",
"#432d79",
"#76d99c",
"#c580db",
"#437535",
"#d84c5f",
"#74d9cf",
"#dc502c",
"#77b4d3",
"#d99639",
"#6c80c2",
"#dac380",
"#312442",
"#c0cfad",
"#833061",
"#528877",
"#8c2e2b",
"#cfb2d2",
"#354027",
"#d57aa7",
"#81793b",
"#45637c",
"#985e31",
"#502725",
"#d49985",
"#8d6570"]

In [53]:
colors = ["#7a61d9",
"#51b950",
"#b340ac",
"#87b03a",
"#d077e3",
"#c1b53a",
"#4a7bdc",
"#e3a53c",
"#6c54ab",
"#db862f",
"#6fa3df",
"#da3d2d",
"#3abbcc",
"#df3952",
"#5fc092",
"#dd4594",
"#3c834d",
"#c73b5e",
"#65752a",
"#a891df",
"#e1652a",
"#496ea8",
"#b03c21",
"#8c5898",
"#a58531",
"#d685bf",
"#aeb16b",
"#a54776",
"#a95f26",
"#e48193",
"#86622f",
"#e76f5b",
"#9b4e5a",
"#dd996f",
"#ad5648"]

In [56]:
colors = ["#51b8b4",
"#c143d5",
"#62c338",
"#6134c0",
"#a0b841",
"#6d68db",
"#cbab3c",
"#5d3385",
"#4dc576",
"#cf369d",
"#51963a",
"#c771d4",
"#336423",
"#d93663",
"#7fb886",
"#e14327",
"#547ac8",
"#da8b37",
"#66add8",
"#c86235",
"#3d6e8c",
"#912d23",
"#437f5e",
"#d46ba3",
"#797a2b",
"#a991cd",
"#3d4820",
"#cd8ea7",
"#b5a76e",
"#53416c",
"#8c6131",
"#802d55",
"#d1967c",
"#67342a",
"#d06869"]

In [35]:
# ok let's figure out how to get these into the plot

In [48]:
# need to make color2, which will be list of lists of what colors should be used per particular macrostate

In [57]:
colors2 = []
dictionary_of_colors = dict()

i = 0
for x in regspace_counters_struct_macro2:
    colors2.append([])
    for key in x:
        if key in dictionary_of_colors:
            color_ = dictionary_of_colors[key]
        else:
            color_ = colors[i]
            i += 1
            dictionary_of_colors[key] = color_
        colors2[-1].append(color_)    

In [58]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("model5_piecharts_absorbed_colorperstrmacro.pdf")

for i in range(len(regspace_counters_struct_macro2)):
    x = regspace_counters_struct_macro2[i]
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%', colors=colors2[i])
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [1]:
# ok so now we want the one color per structural macrostate, but without the closest-8 absorption

In [47]:
regspace_counters_struct_macro

[{33: 100},
 {33: 79, 34: 21},
 {1: 100},
 {9: 100},
 {1: 2, 17: 98},
 {31: 76, 39: 24},
 {1: 54, 9: 46},
 {9: 6, 10: 57, 17: 25, 26: 1, 33: 11},
 {37: 100},
 {23: 24, 31: 76},
 {1: 42, 2: 2, 9: 53, 17: 3},
 {1: 54, 9: 16, 10: 30},
 {9: 1, 18: 41, 25: 58},
 {7: 21, 15: 42, 21: 5, 31: 20, 33: 1, 38: 11},
 {6: 2, 7: 1, 13: 2, 14: 13, 15: 37, 19: 1, 30: 44},
 {3: 2, 9: 26, 10: 51, 11: 3, 17: 4, 24: 2, 30: 7, 35: 5},
 {27: 12, 28: 2, 29: 1, 30: 2, 33: 15, 34: 54, 35: 7, 40: 1, 43: 6},
 {1: 22, 2: 5, 9: 19, 10: 54},
 {34: 55, 35: 18, 36: 1, 37: 3, 41: 16, 43: 1, 45: 6},
 {25: 1, 29: 12, 30: 18, 33: 17, 34: 14, 35: 5, 37: 4, 40: 1, 41: 28}]

In [11]:
# check how many colors we'll need - then repeat everythign I did above for here

In [12]:
test = []

for x in regspace_counters_struct_macro:
    for y in x:
        test.append(y)

In [13]:
len(set(test))

35

In [15]:
# let's make a palette of 50 for this

In [16]:
# this is default preset, 50 colors from iWantHue

In [17]:
colors = ["#e37367",
"#40d17c",
"#9d44ba",
"#50c352",
"#e26cdf",
"#89bb2d",
"#9c6be5",
"#3f9728",
"#d03ea8",
"#80b652",
"#5d75e3",
"#b4b42f",
"#6c52ae",
"#d8ae34",
"#5579bf",
"#d3441d",
"#47c7d0",
"#e33752",
"#5dd0a7",
"#e53e83",
"#49ac6a",
"#a83e94",
"#447d2f",
"#ce81dc",
"#a0a94e",
"#e071b4",
"#30784f",
"#c43a33",
"#53a4d6",
"#ea713f",
"#3ea089",
"#b12e4b",
"#89af72",
"#b13871",
"#5f6924",
"#b198dc",
"#d09028",
"#8c5898",
"#cdb568",
"#974d6f",
"#8b7d33",
"#e087ad",
"#d7833a",
"#df5971",
"#946330",
"#dd8787",
"#ab521e",
"#a44f53",
"#db9c6c",
"#a64c33"]

In [18]:
colors2 = []
dictionary_of_colors = dict()

i = 0
for x in regspace_counters_struct_macro:
    colors2.append([])
    for key in x:
        if key in dictionary_of_colors:
            color_ = dictionary_of_colors[key]
        else:
            color_ = colors[i]
            i += 1
            dictionary_of_colors[key] = color_
        colors2[-1].append(color_)    

In [22]:
cd piecharts

/Users/rafalpwiewiora/repos/MSM/11707_11709_SET8_apo/msm_pipeline/dist/regspace_analysis/piecharts


In [23]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("model5_piecharts_colorperstrmacro.pdf")

for i in range(len(regspace_counters_struct_macro)):
    x = regspace_counters_struct_macro[i]
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%', colors=colors2[i])
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    

In [48]:
# another change - do same color per structural macrostate FOR ALL MODELS - so we have a list of 56 colors
# default preset from iwanthue

In [49]:
colors = ["#aa5f5d",
"#4ecb5f",
"#c45fe3",
"#87bc2d",
"#8b39b6",
"#379f3b",
"#9d5ee0",
"#b9bd40",
"#4c69db",
"#d0b231",
"#8163d6",
"#86be59",
"#ac36a2",
"#678f24",
"#c35ec4",
"#58c086",
"#d13f94",
"#448136",
"#ed77d7",
"#958f24",
"#c07de7",
"#e4a231",
"#704ca8",
"#e6832c",
"#4789dc",
"#e1532f",
"#48b1da",
"#d93640",
"#52bfb0",
"#d3335a",
"#37835d",
"#e3427e",
"#a1bb74",
"#b4509d",
"#b9a753",
"#8f83de",
"#c38734",
"#5666a4",
"#d56c35",
"#96a0e0",
"#a64219",
"#c88acd",
"#5a6322",
"#895191",
"#838948",
"#ac5487",
"#88671f",
"#e488b1",
"#d8af70",
"#8e415d",
"#eb9a6f",
"#be506e",
"#8e552f",
"#e88280",
"#b9744a",
"#ba4846"]

In [50]:
dictionary_of_colors = dict()

for i in range(56):
    dictionary_of_colors[i] = colors[i]

colors2 = []

i = 0
for x in regspace_counters_struct_macro:
    colors2.append([])
    for key in x:
        color_ = dictionary_of_colors[key]
        colors2[-1].append(color_)    

In [51]:
import matplotlib.backends.backend_pdf
pdf = matplotlib.backends.backend_pdf.PdfPages("piecharts/model5_piecharts_colorperstrmacro.pdf")

for i in range(len(regspace_counters_struct_macro)):
    x = regspace_counters_struct_macro[i]
    fig1, ax1 = plt.subplots()
    ax1.pie(list(x.values()), labels=list(x.keys()), autopct='%1.1f%%', colors=colors2[i])
    ax1.axis('equal')
    pdf.savefig(fig1, dpi=300)
    plt.close()
    
pdf.close()    